# R MNIST Model

 * Wrap an R model (using the ```caret``` library) for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
 
## Dependencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)
 * R

```bash
pip install seldon-core
```

## Train locally
 

In [11]:
!rm -f *ubyte && ./get_data.sh && Rscript train.R

--2019-04-24 16:43:55--  http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... 216.165.22.6
Connecting to yann.lecun.com (yann.lecun.com)|216.165.22.6|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9912422 (9.5M) [application/x-gzip]
Saving to: ‘train-images-idx3-ubyte.gz’

train-images-idx3-u 100%[===================>]   9.45M  2.56MB/s    in 4.5s    

2019-04-24 16:43:59 (2.10 MB/s) - ‘train-images-idx3-ubyte.gz’ saved [9912422/9912422]

--2019-04-24 16:43:59--  http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... 216.165.22.6
Connecting to yann.lecun.com (yann.lecun.com)|216.165.22.6|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28881 (28K) [application/x-gzip]
Saving to: ‘train-labels-idx1-ubyte.gz’

train-labels-idx1-u 100%[===================>]  28.20K  --.-KB/s    in 0.08s   

2019-04-24 16:44:00 (346 KB/s) - ‘train-labels-id

Wrap model using s2i

In [12]:
!s2i build . seldonio/seldon-core-s2i-r:0.1 r-mnist:0.1

---> Installing application source...
---> Installing dependencies ...
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cloud.r-project.org/src/contrib/pls_2.7-1.tar.gz'
Content type 'application/x-gzip' length 4390103 bytes (4.2 MB)
downloaded 4.2 MB

* installing *source* package ‘pls’ ...
** package ‘pls’ successfully unpacked and MD5 sums checked
** R
** data
*** moving datasets to lazyload DB
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded
* DONE (pls)

The downloaded source packages are in
	‘/tmp/RtmpSNT0ox/downloaded_packages’
Build completed successfully


In [13]:
!docker run --name "mnist_predictor" -d --rm -p 5000:5000 r-mnist:0.1

5104ca6b9b9b9f8a778c46ae1c9d84a733ea63732dce128066306f7fcc49bb82


Send some random features that conform to the contract

In [14]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p

----------------------------------------
SENDING NEW REQUEST:

[[0.109 0.293 0.668 0.174 0.445 0.31  0.777 0.777 0.366 0.064 0.191 0.17
  0.254 0.693 0.425 0.566 0.443 0.931 0.575 0.524 0.978 0.797 0.42  0.928
  0.246 0.996 0.433 0.438 0.807 0.419 0.567 0.202 0.12  0.158 0.055 0.471
  0.347 0.491 0.988 0.559 0.482 0.922 0.566 0.182 0.19  0.977 0.933 0.541
  0.507 0.708 0.633 0.919 0.578 0.451 0.569 0.08  0.835 0.625 0.376 0.534
  0.18  0.123 0.212 0.186 0.254 0.369 0.93  0.276 0.565 0.738 0.951 0.222
  0.651 0.584 0.549 0.145 0.891 0.65  0.861 0.584 0.573 0.992 0.34  0.252
  0.122 0.998 0.066 0.883 0.606 0.651 0.764 0.591 0.314 0.048 0.269 0.843
  0.116 0.944 0.633 0.639 0.166 0.537 0.115 0.559 0.798 0.646 0.996 0.492
  0.178 0.77  0.107 0.109 0.302 0.559 0.497 0.382 0.35  0.764 0.744 0.75
  0.46  0.507 0.143 0.673 0.123 0.    0.569 0.442 0.485 0.923 0.018 0.16
  0.4   0.99  0.958 0.718 0.373 0.161 0.263 0.989 0.782 0.685 0.363 0.072
  0.177 0.757 0.383 0.628 0.067 0.876 0.78  0.479 0.

In [15]:
!docker rm mnist_predictor --force

mnist_predictor


# Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [ ]:
!minikube start --memory 4096

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

In [ ]:
!helm init

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

In [2]:
!helm install ../../../helm-charts/seldon-core-controller --name seldon-core --set usage_metrics.enabled=true --namespace seldon-system

NAME:   seldon-core
LAST DEPLOYED: Wed Apr 24 16:36:50 2019
NAMESPACE: seldon-system
STATUS: DEPLOYED

RESOURCES:
==> v1/Secret
NAME                                   TYPE    DATA  AGE
seldon-operator-webhook-server-secret  Opaque  0     0s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s

==> v1/ClusterRole
seldon-operator-manager-role  0s

==> v1/ClusterRoleBinding
NAME                                 AGE
seldon-operator-manager-rolebinding  0s

==> v1/Service
NAME                                        TYPE       CLUSTER-IP     EXTERNAL-IP  PORT(S)  AGE
seldon-operator-controller-manager-service  ClusterIP  10.110.57.111  <none>       443/TCP  0s

==> v1/StatefulSet
NAME                                DESIRED  CURRENT  AGE
seldon-operator-controller-manager  1        1        0s

==> v1/Pod(related)
NAME                                  READY  STATUS             RESTARTS  AGE
seldon-operator-contro

In [4]:
!kubectl rollout status statefulset.apps/seldon-operator-controller-manager -n seldon-system

partitioned roll out complete: 1 new pods have been updated...


## Setup Ingress
There are gRPC issues with the latest Ambassador, so we rewcommend 0.40.2 until these are fixed.

In [5]:
!helm install stable/ambassador --name ambassador --set image.tag=0.40.2

NAME:   ambassador
LAST DEPLOYED: Wed Apr 24 16:37:34 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/Deployment
NAME        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
ambassador  3        3        3           0          0s

==> v1/Pod(related)
NAME                         READY  STATUS             RESTARTS  AGE
ambassador-5b89d44544-5rmss  0/1    ContainerCreating  0         0s
ambassador-5b89d44544-kzmp2  0/1    ContainerCreating  0         0s
ambassador-5b89d44544-qmf7z  0/1    ContainerCreating  0         0s

==> v1/ServiceAccount
NAME        SECRETS  AGE
ambassador  1        0s

==> v1beta1/ClusterRole
NAME        AGE
ambassador  0s

==> v1beta1/ClusterRoleBinding
NAME        AGE
ambassador  0s

==> v1/Service
NAME               TYPE          CLUSTER-IP     EXTERNAL-IP  PORT(S)                     AGE
ambassador-admins  ClusterIP     10.98.164.148  <none>       8877/TCP                    0s
ambassador         LoadBalancer  10.111.173.75  <pending>    80:30659/TCP,443

In [6]:
!kubectl rollout status deployment.apps/ambassador

Waiting for deployment "ambassador" rollout to finish: 0 of 3 updated replicas are available...
Waiting for deployment "ambassador" rollout to finish: 1 of 3 updated replicas are available...
Waiting for deployment "ambassador" rollout to finish: 2 of 3 updated replicas are available...
deployment "ambassador" successfully rolled out


In [16]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-r:0.1 r-mnist:0.1

---> Installing application source...
---> Installing dependencies ...
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cloud.r-project.org/src/contrib/pls_2.7-1.tar.gz'
Content type 'application/x-gzip' length 4390103 bytes (4.2 MB)
downloaded 4.2 MB

* installing *source* package ‘pls’ ...
** package ‘pls’ successfully unpacked and MD5 sums checked
** R
** data
*** moving datasets to lazyload DB
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded
* DONE (pls)

The downloaded source packages are in
	‘/tmp/RtmpDwBobW/downloaded_packages’
Build completed successfully


In [17]:
!kubectl create -f r_mnist_deployment.json

seldondeployment.machinelearning.seldon.io/seldon-deployment-example created


Wait until ready (replicas == replicasAvailable)

In [18]:
!kubectl rollout status deploy/r-mnist-deployment-r-mnist-predictor-d12e455

Waiting for deployment "r-mnist-deployment-r-mnist-predictor-d12e455" rollout to finish: 0 of 1 updated replicas are available...
deployment "r-mnist-deployment-r-mnist-predictor-d12e455" successfully rolled out


In [19]:
!seldon-core-api-tester contract.json `minikube ip` `kubectl get svc ambassador -o jsonpath='{.spec.ports[0].nodePort}'` \
    seldon-deployment-example --namespace seldon -p

----------------------------------------
SENDING NEW REQUEST:

[[0.503 0.251 0.133 0.303 0.7   0.267 0.414 0.708 0.152 0.756 0.255 0.517
  0.227 0.109 0.948 0.627 0.129 0.769 0.112 0.707 0.429 0.486 0.358 0.707
  0.566 0.864 0.411 0.022 0.725 0.62  0.204 0.737 0.743 0.073 0.406 0.842
  0.013 0.16  0.842 0.183 0.583 0.146 0.601 0.727 0.072 0.998 0.512 0.552
  0.679 0.91  0.848 0.725 0.414 0.947 0.387 0.595 0.395 0.323 0.864 0.002
  0.616 0.694 0.182 0.026 0.649 0.841 0.136 0.643 0.474 0.105 0.225 0.388
  0.97  0.759 0.541 0.143 0.412 0.603 0.313 0.005 0.057 0.679 0.595 0.119
  0.89  0.439 0.639 0.012 0.783 0.128 0.522 0.758 0.586 0.647 0.075 0.482
  0.573 0.196 0.796 0.539 0.322 0.132 0.6   0.408 0.175 0.099 0.369 0.326
  0.803 0.774 0.317 0.91  0.166 0.894 0.235 0.277 0.329 0.564 0.767 0.359
  0.982 0.413 0.252 0.136 0.084 0.262 0.068 0.887 0.78  0.947 0.542 0.019
  0.036 0.912 0.368 0.392 0.299 0.765 0.048 0.021 0.855 0.275 0.944 0.94
  0.037 0.111 0.09  0.991 0.793 0.345 0.231 0.098 

In [ ]:
!minikube delete